In [38]:
import warnings
warnings.simplefilter("always")
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [39]:
# for saving files later
tutorials_dir = Path.cwd()

Here we use a very slender strips element shape 

In [40]:
# --- Beam geometry ---
L = 20.0   # length in x
H = 1.0    # height in y
nx = 1    # number of elements along length
ny = 2000     # number of elements along height

ele_type = "D2_nn3_tri"  # 2D, 4-node quadrilateral (linear)
ndof = 2                  # 2 DOFs per node (x, y)

In [41]:
# Generate a rectangular mesh
coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
# coords: shape (n_nodes, 2)
# connect: shape (n_nodes_per_elem, n_elems)

# --- Identify boundaries ---
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
    coords, connect, ele_type, x_lower=0.0, x_upper=L, y_lower=0.0, y_upper=H
)

# 1) Clamp the left edge: fix x- and y-displacements = 0
fixed_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)

And we apply a huge load force but just one step.

In [45]:
# 2) Uniform downward traction on the top edge (y=H)
# Let q be negative in the y-direction
q = -10000  # load per unit length in x
# For a 2D plane strain problem, this is a traction (tx, ty) = (0, q)
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, q)

# Combine boundary conditions
fixed_nodes = fixed_left  # only the left edge is clamped

In [46]:
# --- Material properties ---
E = 100000.0
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))

material_props = np.array([mu, kappa])
print(f"Material properties: mu={mu:.3f}, kappa={kappa:.3f}")

Material properties: mu=38461.538, kappa=83333.333


We can see the solver fails to converge

In [47]:
# Number of incremental load steps
nr_num_steps = 1

# --- Solve with your hyperelastic solver ---
displacements_all, nr_info_all = hyperelastic_solver(
    material_props,
    ele_type,
    coords.T,      # shape (2, n_nodes)
    connect.T,     # shape (n_nodes_per_elem, n_elems)
    fixed_nodes,
    dload_info,
    nr_print=True,
    nr_num_steps=nr_num_steps,
    nr_tol=1e-12,
    nr_maxit=30,
)

Step 0, load factor = 1.000
Iteration 1, Correction=1.000000e+00, Residual=1.249375e+01, tolerance=1.000000e-12
Iteration 2, Correction=4.930607e-01, Residual=2.100792e+04, tolerance=1.000000e-12
Iteration 3, Correction=4.831504e-01, Residual=6.169882e+03, tolerance=1.000000e-12
Iteration 4, Correction=4.868074e-01, Residual=1.794021e+03, tolerance=1.000000e-12
Iteration 5, Correction=1.141936e+00, Residual=4.821335e+02, tolerance=1.000000e-12
Iteration 6, Correction=9.453576e-01, Residual=1.369243e+04, tolerance=1.000000e-12
Iteration 7, Correction=1.941406e-01, Residual=5.979526e+03, tolerance=1.000000e-12
Iteration 8, Correction=2.712213e-01, Residual=3.605265e+04, tolerance=1.000000e-12
Iteration 9, Correction=0.000000e+00, Residual=nan, tolerance=1.000000e-12
